In [93]:
from bnlp import NLTKTokenizer
bnltk = NLTKTokenizer()
import pandas as pd
import csv
import string
from sklearn.utils import shuffle
pd.set_option('display.max_colwidth', 255)
from collections import Counter
from tqdm import tqdm
from sklearn import model_selection
from sklearn import metrics
import numpy as np
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline
from bnlp.corpus import stopwords
from bnlp.corpus.util import remove_stopwords
stopwords = stopwords()
path = '../../DeepHateLingo/'

In [94]:
#new multi_hate_data
multi_hate_data = pd.read_csv(f"{path}input/multi_hate_data.csv")
multi_hate_data.label.value_counts(), len(multi_hate_data)

(Personal        1336
 Geopolitical     639
 Religious        254
 Political        188
 Name: label, dtype: int64,
 2417)

In [95]:
# previous 6115 datapoint
bengali_hate_v3 = pd.read_csv(f"{path}input/bengali_hate_v3.0.csv")
bengali_hate_v3.label.value_counts(), len(bengali_hate_v3)

(Personal        2408
 Geopolitical    1849
 Religious       1001
 Political        857
 Name: label, dtype: int64,
 6115)

In [96]:
data = pd.concat([multi_hate_data,bengali_hate_v3],axis = 0).reset_index(drop=True)
data = data.drop_duplicates(subset=['text','label'],keep='first')
data = data.reset_index(drop=True)
data[data.isnull().any(axis=1)]

,text,label


In [97]:
data[data['text'] == 'পড়াশুনা করে মানুষ হবি,আরে না মদ খাবি মানুষ হবি!']

,text,label
711,"পড়াশুনা করে মানুষ হবি,আরে না মদ খাবি মানুষ হবি!",Personal


In [98]:
data.label.value_counts(), len(data)

(Personal        3738
 Geopolitical    2487
 Religious       1255
 Political       1045
 Name: label, dtype: int64,
 8525)

In [99]:
for _ in range(5000):
    data = shuffle(data)
    data = data.reset_index(drop=True)
data.label.value_counts(), len(data)

(Personal        3738
 Geopolitical    2487
 Religious       1255
 Political       1045
 Name: label, dtype: int64,
 8525)

In [100]:
def get_indices(df,col,n):
    tmp = []
    for i in range(len(df)):
        if len(bnltk.word_tokenize(df[col][i])) <= n:
            tmp.append(i)
    return tmp
tmp = get_indices(data, "text", 5)
data = data.drop(tmp)
data = data.reset_index(drop=True)
data.label.value_counts(), len(data)

(Personal        3513
 Geopolitical    2364
 Religious       1211
 Political        999
 Name: label, dtype: int64,
 8087)

In [101]:
train_data = data[: 5887]
valid_data = data[5887 : 6887]
test_data = data[6887 :]

In [102]:
5887 + 1000 + 1200 = 8087

8087

In [103]:
len(train_data),len(valid_data), len(test_data)

(5887, 1000, 1200)

In [104]:
train_data

,text,label
0,৭৫ হামলার আসামী কে এই গডফাদার কালা মন্দির?,Personal
1,রাজাকারদের মন্ত্রী বানানোর দায়ে তার আরও কঠিন বিচার বাকি আছে,Political
2,বৃটিশ নাগরিকদের বাংলাদেশ ভ্রমনে সতর্কতা জারী ২৩ আগ্নেয়াস্ত্রসহ যুবলীগ নেতা গ্রেফতার,Geopolitical
3,বলদ নয়াদিগন্ত কালকে রিপোর্ট করলো হিলারী শপথ নেওয়ার প্রস্তুতি নিচ্ছেন ট্রাম্প শিবিরে হতাশা।,Political
4,অসভ্য বর্বর বেহায়া ভোট ডাকাত সকল প্রকার খারাপ কাজের চ্যাম্পিয়ন শেখ হাসিনার সরকারের বিকল্প আর নেই,Geopolitical
...,...,...
5882,কতটা!জানোয়ার হলে দাদা নামের কুলাঙ্গার তার সাত বছরের নাতনীকে ধর্ষণ করতে পারে,Personal
5883,ঐ শুয়োরের মুখে কোষে দুটা জুতার বারি মার ঐ খানকি মাগির এত সাহস পাইলো কথা থেকে ট্রাম জিতার পর থেকে এই মাগির এত স্পর্ধা বেড়েছে,Personal
5884,"তাকে বলে দিও, লিঙ্গ মুখে নিলে বাচ্চা হয় না",Personal
5885,"হেসে লাভ নাই, এইবার মাইর খাবা। কোন মাফ পাবা না।",Personal


In [105]:
valid_data

,text,label
5887,"বীর্যপাতের আগে সে হিরো, আর পরে সে জিরো",Personal
5888,"এই টাকলা, এদিকে শোন ভদ্র ভাবে কথা বল মাননীয় বাল হীন ব্যক্তি একটু এদিকে শুনবেন?",Personal
5889,তোদের জ¦লবে আমার তাতেই চলবে। কি বুজলি?,Personal
5890,আপনার গরম লাগে না শরম লাগে?,Personal
5891,নিজের আল্লাহ রি চরিত্রটা দেখ ৭২ হুরের লোভে তরা অন্দহয়ে আছিস,Religious
...,...,...
6882,"আমরা লুটে পুটে খাই, তাই আনন্দের সীমা নাই",Geopolitical
6883,"সব মিমারদের চরিত্র খারাপ, তাতে আমার বাল ছিড়া গেছে",Personal
6884,এই ম্যাইয়ারে হেমায়েতপুর থেইকা খোলা অবস্থায় ছাইড়া দিছে কেড়ায়!!,Personal
6885,"অভিজিৎ রায় হত্যাকান্ড। খুনিরা দিব্যি চলে গেল, পুলিশ নীরবে দেখল।",Personal


In [106]:
test_data

,text,label
6887,"মুশরিকদের হত্যা কর যেখানেই তাদের পাও এবং তাদের ধরো, ঘেরাও করো এবং তাদের প্রতিটি ঘাঁটিতে তাদের খবরাখবর নেওয়ার জন্য শক্ত হয়ে বসো।",Religious
6888,কোথায় মহারানী আর কোথায় চুতমারানী?,Personal
6889,হিন্দু বা কাফের দের সাতে হাত মিলিয়ে সারা দুনিয়াতে ফেতনা লাগায়ে নিজেরা শেষ হয়ে জাচ্ছে,Religious
6890,"নিজের তো সব গেছেই এখন মেয়েটার জীবন শেষ করতে আকাটা বিয়া করছে খুব বেশী না, মাস নয় থেকে ছয় মিথিলার হবে পরাজয়",Personal
6891,"এই ছবি পুরো জাতীকে লজ্জা দেয়, আজ ঢাকায় ট্রাক থেকে পেঁয়াজ কিনতে এসে এভাবে ছোট্ট বাচ্চাটাকে কোলে নিয়ে অসহায় চেহারায় টাকা দিচ্ছেন বিক্রেতাকে।",Political
...,...,...
8082,"কুদ্দুস বলতো গরু আমাদের কি দেয়? স্যার, গরু আমাদের গুতা দেয়।",Personal
8083,ওরা দুজন আগের জন্মে মুসলমান ছিল এই জন্মে হিন্দু হয়েও ভুলতে পারেনি তাই মাসতুতো ভাই বোন প্রেম করেছে কারন ভাই বোনের বিবাহ মুসলমানদের মধ্যেই হয়,Personal
8084,মুহম্মদ দুনিয়াতেই মাগিবাজি লম্পটগিরি করল তার উন্মত জন্নতে মাগীবাজি করবে।,Religious
8085,"কেনেন, লাগান, ভুলে যান। আরাম করে চুদে দেন।",Personal


In [107]:
train_data = train_data.reset_index(drop=True)
valid_data = valid_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

In [108]:
test_data[test_data.isnull().any(axis=1)]

,text,label


In [115]:
train_data.label.value_counts(), len(train_data)

(Personal        2561
 Geopolitical    1691
 Religious        908
 Political        727
 Name: label, dtype: int64,
 5887)

In [116]:
valid_data.label.value_counts(), len(valid_data)

(Personal        428
 Geopolitical    313
 Religious       144
 Political       115
 Name: label, dtype: int64,
 1000)

In [117]:
test_data.label.value_counts(), len(test_data)

(Personal        524
 Geopolitical    360
 Religious       159
 Political       157
 Name: label, dtype: int64,
 1200)

In [124]:
def making_label(st):
    if(st=='Personal'):
        return 0
    elif(st=='Political'):
        return 1
    elif(st=='Religious'):
        return 2
    elif(st=='Geopolitical'):
        return 3
train_data['target'] = train_data['label'].apply(making_label)
train_data.target.value_counts(), len(train_data)

(0    2561
 3    1691
 2     908
 1     727
 Name: target, dtype: int64,
 5887)

In [125]:
test_data['target'] = test_data['label'].apply(making_label)
test_data.target.value_counts(), len(test_data)

(0    524
 3    360
 2    159
 1    157
 Name: target, dtype: int64,
 1200)

In [126]:
valid_data['target'] = valid_data['label'].apply(making_label)
valid_data.target.value_counts(), len(valid_data)

(0    428
 3    313
 2    144
 1    115
 Name: target, dtype: int64,
 1000)

In [127]:
train_data.to_csv(f"{path}input/train.csv",index = False)
test_data.to_csv(f"{path}input/test.csv",index = False)
valid_data.to_csv(f"{path}input/validation.csv",index = False)